# Documentation: `AutoDiff`

## Table of Contents
1. [Introduction](#introduction)
2. [How To Use AutoDiff](#how-to)
    - [Installing the Package](#installation)
    - [Demo](#demo)
2. [Background](#background)
4. [Software Organization](#SoftwareOrganization)
    - [Directory Structure](#structure)
    - [Modules](#modules)
    - [Test Suite](#tests)
    - [Distribution](#distribution)
    - [Installation](#installation)
5. [Implementation](#implementation)
    - [Core Data Structure](#p1)
    - [Major Class](#p2)
    - [Method and Name Attributes in AutoDiff Class](#p3)
    - [Other Functions](#p4)
    - [External Dependences](#p5)
6. [Future](#future)

<a id="introduction"></a>
## Introduction 

It goes without saying that taking derivatives is an essential operation in numerical methods, optimization, and science. From a computational perspective, however, calculating a derivative can be a difficult.

If one uses **finite differences** (i.e. $f'(x) \approx (f(x+\epsilon) - f(x))/\epsilon)$), one needs to choose $\epsilon$ appropriately. If $\epsilon$ is too large, the approximation is poor. If $\epsilon$ is too small, one introduces round-off errors.

Alternatively, if one uses **symbolic differentiation** (i.e. an algorithm that produces the derivative as a symbolic function), the problem becomes computationally infeasible when you either have functions with many inputs or want to take high order derivatives. These two scenarios occur often in applications.

**Automatic differentiation** overcomes these challenges by providing both quick and accurate derivatives.

<a id="how-to"></a>
## How To Use `autoder`

<a id="installation"></a>
### Installing the Package
You can use pip to install the test `autoder` package. 

```
pip install -i https://test.pypi.org/simple/ autoder
```

<a id="demo"></a>
### Demo for Automatic Differentiation

To use the package, first import the `variables` and `AD_numpy` modules.

In [24]:
import sys
sys.path.insert(0, '../AutoDiff')
import AutoDiff.variables as v
import AutoDiff.AD_numpy as np

The core object in `autoder` is the `Variable` class. They allow numbers to have both objects and derivatives.

In [18]:
# Initialize a variable with a name and a value
x = v.Variable('x', 3)
y = v.Variable('y', 4)

Functions of `Variable` instances return another `Variable`. Functions such as `sin` and `log` can be found in the `AD_numpy` module.

In [19]:
f = x**2+y**2+np.sin(x)
print(f)

Variable name: f(f(f(x),f(y)),x), Value: 25.14112000805987, Derivatives: {'y': 8, 'x': 5.010007503399555}


To return the partial derivatices, use the `partial_der` function

In [20]:
f.partial_der(x)

5.010007503399555

To return the jacobian, call the `jacobian` function

In [21]:
f.jacobian()

{'y': 8, 'x': 5.010007503399555}

To demonstrate the functionality of this package, here is a simple implementation of gradient descent to find the minimal parameters of $f=(x+3)^2+(y-5)^2$.

In [22]:
# Implementation of gradient descent
function = lambda x, y: (x+3)**2+(y-5)**2
params = [3,4]
print('initial parameters', params)
x = v.Variable('x', params[0])
y = v.Variable('y', params[1])
for epoch in range(1000):
    grad = function(x,y).jacobian()
    for idx, g in enumerate(grad.values()): 
        params[idx] = params[idx] - 0.01*g
    x = v.Variable('x', params[0])
    y = v.Variable('y', params[1])
print('final parameters',params)

initial parameters [3, 4]
final parameters [1393926277.8034523, -1393926275.8034523]


<a id="demo"></a>
### Demo for Optimization

To use the package, first import the Optimizer module.

In [25]:
sys.path.insert(0, '../Implementation')
import Optimizer as op

Optimizer module has various methods for optimizations, such as min_conjugate_gradient, min_steepestgradient, and min_BFGS. 

In [34]:
print(dir(op))

['MAXITER', 'PRECISION', 'Result', 'Variable', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__', '_get_grad', '_line_search', '_update_hessian', 'findroot', 'min_BFGS', 'min_conjugate_gradient', 'min_gradientdescent', 'min_newton', 'min_steepestdescent', 'minimize', 'np', 'root_BFGS', 'root_SGD', 'root_gradientdescend', 'root_newton', 'root_secant_method', 'time']


Once choosing which optimization method to use, a user can initialize the function and its initial values. 

In [28]:
# an example function
f = lambda x, y, z: (x-4)**2 + (y-3)**2 + (z-2)**2
op.min_conjugate_gradient(f, [3, 2, 1], 1e-6, 50)

To demonstrate the functionality of this package, here is a simple implementation of min_conjugate_gradient for $(x-4)^2+ (y-3)^2 + (z-2)^2$

## <font color='red'>We can display the result of min_conjugate_gradient, as an example result and what it shows.</font>

<a name="background"></a>
## Background

Automatic differentiation computes derivatives recursively using the chain rule. All functions are either an **"elementary" function**, for which we know the derivative, or a composition of elementary functions. To calculate the derivative of a composite function $f(g(x))$, we apply the chain rule as follows:

$$
\frac{df}{dx} = \frac{df}{dg}\frac{dg}{dx}
$$

The chain rule can be applied recursively, which we exploit in automatic differentiation. For example, if we have a complex composite function $f(g(h(x)))$, we can compute f'(x) by first calculating

$$
\frac{dg}{dx} = \frac{dg}{dh}\frac{dh}{dx}
$$

and then plugging this derivative into 

$$
\frac{df}{dx} = \frac{df}{dg}\frac{dg}{dx}
$$

This is in fact a simple example of **forward-mode** automatic differentiation. In general, to conduct forward mode automatic differentiation, we represent our function to differentiate as a **computational graph**. The computational graph  captures the inputs and outputs of our elementary functions. In an example that can be found [here](http://www.columbia.edu/~ahd2125/post/2015/12/5/), we can represent $f(x,y)=\cos(x)\sin(y)+\frac{x}{y}$ as 

![comp-graph](figs/comp_graph_background.png)

By computing derivatives recursively using the chain rule from the inputs $x$ and $y$ to the output $f$, we can calculate the derivative over the entire graph.

This project will implement only forward-mode automatic differentiation, but as an aside, **reverse-mode automatic differentiation** begins at the output(s) of the computational graph and calculates derivates using the chain rule by traversing the graph backwards.

In terms of optimization, we implemented advanced gradient-based optimization methods such as conjugate gradient, steepest descent, and BFGS and the basic gradient descent, in order to fully utilize properties of our automatic differentiation package. The general idea to use forward-mode automatic differentiation to find out the minimum of a function in different ways. Given initial values and a sample function, 
initial values are updated in every iteration based on the different optimization methods, slowly moving toward the mininum value. 

For example, an illustration below shows how steepest descent and conjugate gradient find optimum by updating the functionfor each step. Details of algorithms are discussed in our optimization package, with appropriate examples in test files. 

![descent.png](figs/descent.png)

<a name="SoftwareOrganization"></a>
## Software Organization  <font color='red'>Will be updated!</font>

<a name="structure"></a>
### Directory Structure

The directory structure will be as follows

`
AutoDiff
|-README.md
|-LICENSE
|-setup.py
|-requirements.txt
|-AutoDiff
  |-__init__.py
  |-variables.py
  |-AD_numpy.py
  |-user_func.py
|-docs
  |-documentation.md
|-tests
  |-__init__.py
  |-test_variables.py
  |-test_numpy.py
  |-test_user_func.py
  |-test_derivatives.py
`

<a name="modules"></a>
### Modules

The `variables` module contains the functionality to define variables that are compatible with automatic differentiation. These variables will be passed to functions in `AD_numpy` or to functions specified by the user with `unary_user_function` or `binary_user_function`.

The `AD_numpy` module stores our mathematical functions that will overwrite the typically used numpy package such that the user can use mathematical functions on Variable instances just as they would with numeric values. 

The `vector_variables` module contains the functionality to wrap vector-valued functions to make them compatible with the Variable instances. This will allow the user to extract the value and jacobian of these vector-valued functions.

The `Optimizer` module introduces various optimization methods implemented based on the `variables` module and the `AD_numpy` module. Each optimization method has function, initial values, type of norm to use, and the precision and maximum number of iterations that determine the stopping point as arguments. 

<a name="tests"></a>
### Test Suite

We will store our tests in the `tests` module and run them using `TravisCI`.

<a name="distribution"></a>
### Distribution <font color='red'>Will be updated!</font>

The package will eventually be distributed on `PyPI`. However, it is currently only available on `test.pypi.org` to indicate that the package is not ready for wide distribution.

<a name="installation"></a>

### Installation

You can use pip to install the test `autoder` package with the following command.

```
pip install -i https://test.pypi.org/simple/ autoder
```


<a name="implementation"></a>
## Implementation 
<a id="p1"></a>

### Major data structure: Variables and the Computational Graph

Our variables will be the nodes in our computational graph. The variables will keep track of the node's value and it's derivative.

<a id="p2"></a>
### Classes

The main class that we will implement is the `Variable` class. All auto-differentiable functions will have inputs and outputs consisting of instances of the `Variable` class. The `Variable` class will be an extension on ordinary python numbers. It will store the derivative of the variable and it's actual value. The `vector_Variable` class extends upon the `Variable` class to allow for users to apply vector-valued functions on the `Variable` class through a simple wrapper function `vectorize_variable`. 

In addition, as a main functionality of our package is to allow for basic optimization that comes with ready-made visualizations, the package comes with a `Result` class that stores the key information of an optimization. These are compatible with our optimization visualization and may also be accessed by the user for their own use. 

<a id="p3"></a>
### Attributes and Methods in Variable Class
* Attributes

The `Variable` class will have three main instance attributes, the name, the value, and the derivative of the variable instance.

`Variable.name`: name of the variable. When the user first instantiates a `Variable` instance, it is important that the user sets the name of the variable. This is necessary to keep track of the various variables while we calculate compound functions. Importantly, this will allow us to calculate the partial derivatives with respect to each of the instantiated `Variable` instances correctly.

`Variable.val`: value of the variables. The shape is the same as the input variable. 

`Variable.der`: value of the derivatives. The derivatives are held in a dictionary, with each key corresponding to the names of base Variable instances that we instantiated. The corresponding value pair is the partial derivative of the function with respect to the key.

* Dunder Methods

In order to override the four basic operations of elementary arithmetic (addition, subtraction, multiplication, and division), we use dunder methods within our `Variable` class. The dunder methods return new `Variable` instances with the updated value and derivatives. We also override the power and negation operations. In addition, we included the equality dunder method (and non-equality method) to allow the user to verify if two Variable instances are equal. They are equal if they share both the same value and Jacobian. 

* Methods

The two main methods that the user will typically use are `Variable.jacobian()` and `Variable.partial_der(x)`. The former returns the jacobian of the function that the user calcluated from `Variable` instances. The latter returns the partial derivative with respect to a specific `Variable` instance. It will throw an error if the specific `Variable` instance is not defined or not present within the function.

<a id="p3"></a>
### Attributes and Methods in vector_Variable Class
* Attributes

The `vector_Variable` class has three main instance attributes, the original vector of Variables, and the value and jacobian of the vector function.

`vector_Variable.variables`: the original vector of Variables.

`vector_Variable.val`: value of the vector function. This is an array that is basically the output of the vector function without using the Variable class.

`vector_Variable.der`: jacobian of the vector function. This is stored in a pandas dataframe to allow the user to identify with Variable each value in the jacobian matrix is calculated with respect to. This can easily be transformed to an array with .values

* Dunder methods

Similar to the basic `Variable` class, we override the four basic operations of elementary arithmetic with dunder methods within our `vector_Variable` class.

* Methods

Similar to the basic `vector_Variable` class, the two main methods that the user will typically use are `vector_Variable.jacobian()` and `vector_Variable.partial_der(x)`. They work similarly as in the `Variable` class. This ensures that our functions are consistent across both Variable classes, making it more intuitive for users.

<a id="p4"></a>
### Attributes in Result Class
* Attributes

The `Result` class is a simple storage class that stores 4 main attributes.

`Result.x`: the value that minimizes the function if the optimizer converges, else the last recorded value before the optimizer terminates due to it reaching the maximum number of iterations

`Result.val_rec`: an array storing the history of values for the optimizer that stores a value for each iteration of the chosen algorithm

`Result.time_rec`: an array storing the history of time taken for each iteration of the chosen algorithm

`Result.converge`: a boolean indicating whether the chosen algorithm converged, thus letting the user know if the value stored in `Result.x` is the value that minimizes the function


<a id="p5"></a>
### Mathematical functions

* Define elementary differentiation function. 

In order to deal with the other elementary functions (exponential, logarithm, powers, roots, trigonometric functions, inverse trigonometric functions, hyperbolic functions, etc.), we will override the numpy elementary functions such that we can use it for our AutoDiff class. 

>For example, we will override the `np.sin` function such that if you use it on an `Variable` instance `x` at a given value, it will return another `Variable` instance with the value of $\sin(x)$, and the calculated derivative of $\dot{x}\cos(x)$ at the given value. Similarly, we will override the `np.exp` function such that if you use it on an `Variable` instance `x` at a given value, it will return another `Variable` instance with the value of $\exp(x)$, and the calculated derivative of $\dot{x}\exp(x)$ at the given value.

<a id="p6"></a>
### Optimizer functions <font color='red'>Will be updated!</font>

We introduce 4 main algorithms for optimization - gradient descent, steepest descent, conjugate gradient method, and Broyden-Fletcher-Goldfarb-Shanno (BFGS) method. The user interacts with a `minimize` function and is able to call upon any of the 4 methods. The result of the optimizer is stored in a `Result` class. The user can then call methods on the `Result` class to make some simple visualization plots easily. 

<a id="p7"></a>
### User defined functions

All the mathematical functions that come with our `autoder` package are implemented with two user-defined functions, one for unary functions, one for binary functions. These two functions not only allow us to streamline future updates for our mathematical functions, but also allow anyone using the package to define their own mathematical functions easily. We believe this is a neat addition that can simplify the work for our users. 

The user might have to reuse a specific complex function several times in their work. They can now implement it as a single function that can be applied to our Variable class easily. A simple example is the secant function. The secant function is just the reciprocal of the sine function, and as such, the user can easily use our sine function for their calculations. However, the user can also create a secant function for their usage as well. 

```python
>>> sec = lambda x: 1/np.cos(x)
>>> sec_der = lambda x: sec(x)*np.tan(x)
>>> ad_sec = unary_user_function(sec, sec_der)
>>> a = Variable('a', 2)
>>> x = ad_sec(a)
>>> x.val
-2.4029979617223809
>>> x.jacobian()
5.25064633769958
```

<a id="p7"></a>
### External dependencies 

The main and only external dependency for our package is the external numpy package. This will be specified in our setup.py file as a dependency that should be installed together with our package. The external numpy package is necessary for our the various mathematical operations necessary for Automatic Differentiation.


<a name="future"></a>
## Future
<a id="p1"></a>

Our `autoder` package allows the user to work with scalar with single inputs and vector functions of multiple inputs. We have also included a basic functionality for the user to work with vector-valued functions and extract the jacobian of the vector of such functions easily. We would like to eventually expand upon these vector-valued functions to allow users to also extract the Hessian of the jacobian matrices, and perhaps even 2nd order derivatives. This will allow us to expand the optimization methods in our package to include methods such as Newton's method, Quasi-Newton's method, etc.

A related future update could also allow users to work with matrix calculus, which has applications in calculating gradients of complex deep learning networks for loss minimzation. Matrix calculus also has substantial applications in statistics and econometrics, such as with regard to the analysis of multivariate normal, or generally, elliptical contoured distributions. Matrix calculus could also be useful in mixture models, such as when one is trying to use an EM algorithm to analyse Gaussian Mixture Models.